# **(RETAIL SALES ETL NOTEBOOK)**

## Objectives

The objective of this notebook is to perform Extract, Transform, Load (ETL) operations on retail sales data to prepare it for analysis.


## Inputs

- Raw retail sales dataset stored in dataset/raw-data/
- Data fields: Weekly sales, store info, promotional markdowns, holidays, etc.
- Python libraries: pandas, numpy

## Outputs

- Cleaned and transformed dataset saved to dataset/clean-data/cleaned_sales_data.csv
- Engineered features like Total_MarkDown

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/isaacola/Documents/vscode-project/retail-sales/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/isaacola/Documents/vscode-project/retail-sales'

# Section 1

## Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Section 2

## ETL Process

In this section, we perform Extract, Transform, Load (ETL) operations on the retail sales data:

- **Extract**: Load raw CSV files (sales, stores, features).
- **Transform**: Merge datasets, handle missing values, convert data types, engineer features.
- **Load**: Save the cleaned dataset to the clean-data directory.

In [11]:
# Extract: Load raw data
sales_df = pd.read_csv('dataset/raw-data/sales-data-set.csv')
stores_df = pd.read_csv('dataset/raw-data/stores-data-set.csv')
features_df = pd.read_csv('dataset/raw-data/Features-data-set.csv')
# Display the first few rows of each DataFrame
sales_df.head()
stores_df.head()
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
# Transform: Merge datasets
# Merge sales with stores
merged_df = pd.merge(sales_df, stores_df, on='Store', how='left')

In [5]:
# Merge sales with stores
merged_df = pd.merge(sales_df, stores_df, on='Store', how='left')

# Merge with features on Store and Date
merged_df = pd.merge(merged_df, features_df, on=['Store', 'Date'], how='left')
merged_df

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,05/02/2010,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,12/02/2010,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,19/02/2010,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,26/02/2010,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,05/03/2010,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,28/09/2012,508.37,False,B,118221,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,False
421566,45,98,05/10/2012,628.10,False,B,118221,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,192.170412,8.667,False
421567,45,98,12/10/2012,1061.02,False,B,118221,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,192.327265,8.667,False
421568,45,98,19/10/2012,760.01,False,B,118221,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,192.330854,8.667,False


In [8]:
# Convert Date to datetime
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%d/%m/%Y')
merged_df['Date']

0        2010-02-05
1        2010-02-12
2        2010-02-19
3        2010-02-26
4        2010-03-05
            ...    
421565   2012-09-28
421566   2012-10-05
421567   2012-10-12
421568   2012-10-19
421569   2012-10-26
Name: Date, Length: 421570, dtype: datetime64[ns]

In [9]:
# Handle missing values: Replace 'NA' with NaN and fill MarkDowns with 0
merged_df.replace('NA', np.nan, inplace=True)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
merged_df[markdown_cols] = merged_df[markdown_cols].fillna(0)
merged_df[markdown_cols]

,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
0,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...
421565,4556.61,20.64,1.50,1601.01,3288.25
421566,5046.74,0.00,18.82,2253.43,2340.01
421567,1956.28,0.00,7.89,599.32,3990.54
421568,2004.02,0.00,3.18,437.73,1537.49


In [10]:
# Feature engineering: Total MarkDown
merged_df['Total_MarkDown'] = merged_df[markdown_cols].sum(axis=1)
merged_df['Total_MarkDown']

0            0.00
1            0.00
2            0.00
3            0.00
4            0.00
           ...   
421565    9468.01
421566    9659.00
421567    6554.03
421568    3982.42
421569    5247.26
Name: Total_MarkDown, Length: 421570, dtype: float64

In [6]:
merged_df.info() # Displays a concise summary of the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday_x   421570 non-null  bool   
 5   Type          421570 non-null  object 
 6   Size          421570 non-null  int64  
 7   Temperature   421570 non-null  float64
 8   Fuel_Price    421570 non-null  float64
 9   MarkDown1     150681 non-null  float64
 10  MarkDown2     111248 non-null  float64
 11  MarkDown3     137091 non-null  float64
 12  MarkDown4     134967 non-null  float64
 13  MarkDown5     151432 non-null  float64
 14  CPI           421570 non-null  float64
 15  Unemployment  421570 non-null  float64
 16  IsHoliday_y   421570 non-null  bool   
dtypes: bool(2), float64(10), int64(3), object(2)
mem

In [7]:
merged_df.shape  # (rows, columns)

(421570, 17)

In [ ]:
# Ensure clean-data directory exists
os.makedirs('dataset/clean-data', exist_ok=True)

In [ ]:
# Load: Save cleaned data
merged_df.to_csv('dataset/clean-data/cleaned_sales_data.csv', index=False)

print("ETL completed. Cleaned data saved to dataset/clean-data/cleaned_sales_data.csv")